In [1]:
# Tensorflow
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.optimizers import legacy
from tensorflow.keras.backend import clear_session
from tensorflow.python.ops import rnn, rnn_cell
from tensorflow.keras.callbacks import TensorBoard
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import time


# Sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import precision_score, recall_score


import matplotlib.pyplot as plt
import seaborn as sns
import os
import ast
import pandas as pd
import numpy as np

# from google.colab import drive

In [2]:
tf.keras.backend.clear_session()

In [3]:
%run Architectures.ipynb

# Possible Hiperparameters

In [4]:
# Hiperparâmetros a variar
epochs_list = [5, 10, 15]
batch_size_list = [32, 64, 128]
learning_rates = [0.0001, 0.0005, 0.001]
momentums = [0.8, 0.9, 0.95]
hidden_units_list = [64, 128, 256]
l2_lambdas = [0.0001, 0.001, 0.01]

n_epochs = 5 
batch_size = 64  
learning_rate = 0.001  
momentum = 0.9  
n_hidden_units = 128  
l2_lambda = 0.0001  

In [5]:

# Defina os objetos HParam aqui
HP_NUM_EPOCHS = hp.HParam('epochs', hp.Discrete(epochs_list))
HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete(batch_size_list))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete(learning_rates))
HP_MOMENTUM = hp.HParam('momentum', hp.Discrete(momentums))
HP_HIDDEN_UNITS = hp.HParam('hidden_units', hp.Discrete(hidden_units_list))
HP_L2_LAMBDA = hp.HParam('l2_lambda', hp.Discrete(l2_lambdas))

# Best Hiperparameter for test

In [6]:
def best_hiperparameter_test(nn_type, mfcc, n_epochs, batch_size, learning_rate, momentum, l2_lambda, n_hidden_units, tensorboard_callback):

    i = 1 #fold1 used for test

    # Define test set paths
    X_test_path = 'UrbanSound8K/audio/fold'+str(i)+'_4sec_mfccs_13/3D_array.npy'
    Y_test_path = 'UrbanSound8K/audio/fold'+str(i)+'_label/3D_array.npy'

    X_val_path1 = 'UrbanSound8K/audio/fold'+str((i+1-1) % 10 + 1)+f'_4sec_mfccs_{mfcc}/3D_array.npy'
    Y_val_path1 = 'UrbanSound8K/audio/fold'+str((i+1-1) % 10 + 1)+'_label/3D_array.npy'
    X_val_path2 = 'UrbanSound8K/audio/fold'+str((i+2-1) % 10 + 1)+f'_4sec_mfccs_{mfcc}/3D_array.npy'
    Y_val_path2 = 'UrbanSound8K/audio/fold'+str((i+2-1) % 10 + 1)+'_label/3D_array.npy'

    # Combine the validation paths
    X_val_paths = [X_val_path1, X_val_path2]
    Y_val_paths = [Y_val_path1, Y_val_path2]

    # # Define training set paths (all remaining folds)
    X_train_paths = ['UrbanSound8K/audio/fold'+str((j-1) % 10 + 1)+f'_4sec_mfccs_{mfcc}/3D_array.npy' for j in range(i+3, i+10)]
    Y_train_paths = ['UrbanSound8K/audio/fold'+str((j-1) % 10 + 1)+'_label/3D_array.npy' for j in range(i+3, i+10)]

    # Define test set paths
    X_test_path = f'UrbanSound8K/audio/fold{i}_4sec_mfccs_{mfcc}/3D_array.npy'
    Y_test_path = f'UrbanSound8K/audio/fold{i}_label/3D_array.npy'

    # Define validation set paths (wrapping around if i+2 > 10)
    X_val_path1 = f'UrbanSound8K/audio/fold{((i+1-1) % 10 + 1)}_4sec_mfccs_{mfcc}/3D_array.npy'
    Y_val_path1 = f'UrbanSound8K/audio/fold{((i+1-1) % 10 + 1)}_label/3D_array.npy'
    X_val_path2 = f'UrbanSound8K/audio/fold{((i+2-1) % 10 + 1)}_4sec_mfccs_{mfcc}/3D_array.npy'
    Y_val_path2 = f'UrbanSound8K/audio/fold{((i+2-1) % 10 + 1)}_label/3D_array.npy'

    # Combine the validation paths
    X_val_paths = [X_val_path1, X_val_path2]
    Y_val_paths = [Y_val_path1, Y_val_path2]

    # Define training set paths (all remaining folds)
    X_train_paths = [f'UrbanSound8K/audio/fold{((j-1) % 10 + 1)}_4sec_mfccs_{mfcc}/3D_array.npy' for j in range(i+3, i+11)]
    Y_train_paths = [f'UrbanSound8K/audio/fold{((j-1) % 10 + 1)}_label/3D_array.npy' for j in range(i+3, i+11)]

    # Load the datasets from the paths
    X_test = np.load(X_test_path)
    Y_test = np.load(Y_test_path)


    X_val1 = np.load(X_val_path1)
    X_val2 = np.load(X_val_path2)
    Y_val1 = np.load(Y_val_path1)
    Y_val2 = np.load(Y_val_path2)

    X_train = [np.load(path) for path in X_train_paths]
    Y_train = [np.load(path) for path in Y_train_paths]

    
    # Find the minimum size among all folds
    min_size = min([X_test.shape[0], X_val1.shape[0], X_val2.shape[0]] + [x.shape[0] for x in X_train])

    # Resize the data of each fold to the minimum size
    X_test_resized = X_test[:min_size]
    Y_test_resized = Y_test[:min_size]

    X_val1_resized = X_val1[:min_size]
    Y_val1_resized = Y_val1[:min_size]

    X_val2_resized = X_val2[:min_size]
    Y_val2_resized = Y_val2[:min_size]

    X_train_resized = [x[:min_size] for x in X_train]
    Y_train_resized = [y[:min_size] for y in Y_train]


    # Combine the validation sets
    X_test = X_test_resized
    Y_test = Y_test_resized
    X_val = np.concatenate((X_val1_resized, X_val2_resized), axis=2)
    Y_val = np.concatenate((Y_val1_resized, Y_val2_resized), axis=2)
    X_train = np.concatenate((X_train_resized[0], X_train_resized[1], X_train_resized[2], X_train_resized[3], X_train_resized[4], X_train_resized[5], X_train_resized[6]), axis=2)
    Y_train = np.concatenate((Y_train_resized[0], Y_train_resized[1], Y_train_resized[2], Y_train_resized[3], Y_train_resized[4], Y_train_resized[5], Y_train_resized[6]), axis=2)

    X_test = X_test.transpose(2,1,0)
    Y_test = Y_test.transpose(2,1,0)
    X_val = X_val.transpose(2,1,0)
    Y_val = Y_val.transpose(2,1,0)
    X_train = X_train.transpose(2,1,0)
    Y_train = Y_train.transpose(2,1,0)


#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

    
    clear_session()  # Clear the previous model
    

    # Call de neural network
    input_shape = (751, mfcc)
    num_classes = 10

    #Neural Network Initialization
    if nn_type == "mlp":
        model = build_mlp_model(input_shape, num_classes, l2_lambda, n_hidden_units)

    elif nn_type == "rnn":
        model = build_rnn_model(input_shape, num_classes, l2_lambda, n_hidden_units)

    else:
        print("Introduce a valid neural network")





    model.summary() # print The neural network's architecture

    # Define the optimizer
    optimizer = legacy.SGD(learning_rate=learning_rate, momentum=momentum)

    #output layer
    model.add(layers.Dense(10, activation='softmax'))

    Y_train = np.squeeze(Y_train, axis=1)
    Y_val = np.squeeze(Y_val, axis=1)   



    # Compile the model.
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )


    history = model.fit(

        #training dataset
        X_train,
        Y_train,

        epochs=n_epochs,  # Number of epochs
        batch_size=batch_size, # Number of samples per batch

        #validation dataset
        validation_data=(X_val, Y_val),


        #callbacks
        callbacks=[tensorboard_callback]
    )

    # test_accuracy = model.evaluate(X_test, Y_test, verbose=0)[1]

    Y_train_classes = np.squeeze(Y_train).argmax(axis=-1) if Y_train.ndim == 3 else Y_train
    Y_val_classes = np.squeeze(Y_val).argmax(axis=-1) if Y_val.ndim == 3 else Y_val

    # Generate predictions
    Y_pred = model.predict(X_test)  # Predicted probabilities

    # Convert predictions to class labels
    Y_pred_classes = np.argmax(Y_pred, axis=-1)  # Convert probabilities to class labels

    # Flatten the one-hot encoded labels to 1D if they are 3D for Y_test
    Y_test_classes = np.squeeze(Y_test).argmax(axis=-1) if Y_test.ndim == 3 else Y_test

    # Make sure the confusion matrix function is correctly imported
    from sklearn.metrics import confusion_matrix

    # Calculate the confusion matrix
    conf_matrix = confusion_matrix(Y_test_classes, Y_pred_classes)

    # Evaluate the model, ensuring the Y_test used here matches the format expected by the model
    scores = model.evaluate(X_test, np.squeeze(Y_test), verbose=0)
    # Calculate precision and recall
    precision = precision_score(Y_test_classes, Y_pred_classes, average='macro')
    recall = recall_score(Y_test_classes, Y_pred_classes, average='macro')

    # Registre os hiperparâmetros e métricas no TensorBoard
    hparams = {
        'epochs': n_epochs,
        # 'batch_size': batch_size,
        # 'learning_rate': learning_rate,
        # 'momentum': momentum,
        # 'n_hidden_units': n_hidden_units,
        # 'l2_lambda': l2_lambda
    }
    run_name = f"run_{int(time.time())}"
    with tf.summary.create_file_writer(f'logs/hparam_tuning/{run_name}').as_default():
        hp.hparams(hparams)  # Registre os valores dos hiperparâmetros
        tf.summary.scalar('accuracy', scores[1], step=1)
        tf.summary.scalar('precision', precision, step=1)
        tf.summary.scalar('recall', recall, step=1)

    # Return the accuracy and the history
    return scores[1], history
    

#### The best number of epochs for MLP with mfccs=13

In [11]:


best_score = 0
best_params = {}
best_history = None

for n_epochs in epochs_list:
   
    #Create a unique directory for each run
    log_dir = "/Users/larasousa/Desktop/uni/ACII/Project_1/best_individual_hiperparameters_13/" + f"run_test_{n_epochs}_{int(time.time())}"
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Pass the tensorboard callback to your training function
    test_accuracy, history = best_hiperparameter_test("mlp", 13, n_epochs, batch_size, learning_rate, momentum, l2_lambda, n_hidden_units, tensorboard_callback)

    # Update best score, parameters, and history if current run is better
    if test_accuracy > best_score:
        best_score = test_accuracy
        best_params = {'n_epochs': n_epochs}
        best_history = history



print("The best Hyperparameters: ", best_params)
print("The best score:", best_score)


                 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 128)               1249792   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 10)                1

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 128)               1249792   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 10)                1

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 128)               1249792   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 10)                1

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### The best batch size for MLP with mfccs=13

In [12]:

# Melhor pontuação, parâmetros e histórico
best_score = 0
best_params = {}
best_history = None

for batch_size in batch_size_list:
   
    #Create a unique directory for each run
    log_dir = "/Users/larasousa/Desktop/uni/ACII/Project_1/best_individual_hiperparameters_13/" + f"run_test{batch_size}_{int(time.time())}"
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Pass the tensorboard callback to your training function
    test_accuracy, history = best_hiperparameter_test("mlp", 13, n_epochs, batch_size, learning_rate, momentum, l2_lambda, n_hidden_units, tensorboard_callback)

    # Update best score, parameters, and history if current run is better
    if test_accuracy > best_score:
        best_score = test_accuracy
        best_params = {'batch_size': batch_size}
        best_history = history


# Exibir os melhores hiperparâmetros e pontuação
print("The best Hyperparameters: ", best_params)
print("The best score:", best_score)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 128)               1249792   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 10)                1

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 128)               1249792   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 10)                1

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 128)               1249792   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 10)                1

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### The best learning rate for MLP with mfccs=13

In [13]:

# Melhor pontuação, parâmetros e histórico
best_score = 0
best_params = {}
best_history = None

for learning_rate in learning_rates:
   
    #Create a unique directory for each run
    log_dir = "/Users/larasousa/Desktop/uni/ACII/Project_1/best_individual_hiperparameters_13/" + f"run_test{learning_rate}_{int(time.time())}"
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Pass the tensorboard callback to your training function
    test_accuracy, history = best_hiperparameter_test("mlp", 13, n_epochs, batch_size, learning_rate, momentum, l2_lambda, n_hidden_units, tensorboard_callback)

    # Update best score, parameters, and history if current run is better
    if test_accuracy > best_score:
        best_score = test_accuracy
        best_params = {'learning_rate': learning_rate}
        best_history = history


# Exibir os melhores hiperparâmetros e pontuação
print("The best Hyperparameters: ", best_params)
print("The best score:", best_score)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 128)               1249792   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 10)                1

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 128)               1249792   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 10)                1

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 128)               1249792   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 10)                1

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### The best Momentum for MLP with mfccs=13

In [14]:


best_score = 0
best_params = {}
best_history = None

for momentum in momentums:
   
    #Create a unique directory for each run
    log_dir = "/Users/larasousa/Desktop/uni/ACII/Project_1/best_individual_hiperparameters_13/" + f"run_test{momentum}_{int(time.time())}"
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Pass the tensorboard callback to your training function
    test_accuracy, history = best_hiperparameter_test("mlp", 13, n_epochs, batch_size, learning_rate, momentum, l2_lambda, n_hidden_units, tensorboard_callback)

    # Update best score, parameters, and history if current run is better
    if test_accuracy > best_score:
        best_score = test_accuracy
        best_params = {'momentum': momentum}
        best_history = history



print("The best Hyperparameters: ", best_params)
print("The best score:", best_score)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 128)               1249792   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 10)                1

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 128)               1249792   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 10)                1

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/15
48/48 [==============================] - 2s 23ms/step - loss: 2.3621 - accuracy: 0.1133 - val_loss: 2.3408 - val_accuracy: 0.1131
Epoch 2/15
48/48 [==============================] - 1s 14ms/step - loss: 2.3467 - accuracy: 0.1160 - val_loss: 2.3296 - val_accuracy: 0.1114
Epoch 3/15
48/48 [==============================] - 1s 16ms/step - loss: 2.3358 - accuracy: 0.1160 - val_loss: 2.3214 - val_accuracy: 0.1103
Epoch 4/15
48/48 [==============================] - 1s 18ms/step - loss: 2.3278 - accuracy: 0.1161 - val_loss: 2.3159 - val_accuracy: 0.1103
Epoch 5/15
48/48 [==============================] - 1s 16ms/step - loss: 2.3223 - accuracy: 0.1161 - val_loss: 2.3124 - val_accuracy: 0.1103
Epoch 6/15
48/48 [==============================] - 1s 17ms/step - loss: 2.3188 - accuracy: 0.1161 - val_loss: 2.3102 - val_accuracy: 0.1103
Epoch 7/15
48/48 [==============================] - 1s 17ms/step - loss: 2.3167 - accuracy: 0.1161 - val_loss: 2.3087 - val_accuracy: 0.1103
Epoch 8/15
48

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### The best Number of Hidden Units for MLP with mfccs=13

In [16]:


best_score = 0
best_params = {}
best_history = None

for n_hidden_units in hidden_units_list:
   
    #Create a unique directory for each run
    log_dir = "/Users/larasousa/Desktop/uni/ACII/Project_1/best_individual_hiperparameters_13/" + f"run_test{n_hidden_units}_{int(time.time())}"
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Pass the tensorboard callback to your training function
    test_accuracy, history = best_hiperparameter_test("mlp", 13, n_epochs, batch_size, learning_rate, momentum, l2_lambda, n_hidden_units, tensorboard_callback)

    # Update best score, parameters, and history if current run is better
    if test_accuracy > best_score:
        best_score = test_accuracy
        best_params = {'n_hidden_units': n_hidden_units}
        best_history = history



print("The best Hyperparameters: ", best_params)
print("The best score:", best_score)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 64)                624896    
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 10)                6

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 128)               1249792   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 10)                1

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/15
48/48 [==============================] - 2s 35ms/step - loss: 2.4343 - accuracy: 0.1170 - val_loss: 2.4505 - val_accuracy: 0.1103
Epoch 2/15
48/48 [==============================] - 1s 25ms/step - loss: 2.4337 - accuracy: 0.1158 - val_loss: 2.4263 - val_accuracy: 0.1103
Epoch 3/15
48/48 [==============================] - 1s 23ms/step - loss: 2.4128 - accuracy: 0.1158 - val_loss: 2.4065 - val_accuracy: 0.1103
Epoch 4/15
48/48 [==============================] - 1s 23ms/step - loss: 2.3973 - accuracy: 0.1158 - val_loss: 2.3916 - val_accuracy: 0.1103
Epoch 5/15
48/48 [==============================] - 1s 23ms/step - loss: 2.3861 - accuracy: 0.1158 - val_loss: 2.3822 - val_accuracy: 0.1103
Epoch 6/15
48/48 [==============================] - 1s 22ms/step - loss: 2.3789 - accuracy: 0.1158 - val_loss: 2.3752 - val_accuracy: 0.1103
Epoch 7/15
48/48 [==============================] - 1s 21ms/step - loss: 2.3735 - accuracy: 0.1158 - val_loss: 2.3707 - val_accuracy: 0.1103
Epoch 8/15
48

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### The best L2_lambda for MLP with mfccs=13

In [17]:


best_score = 0
best_params = {}
best_history = None

for l2_lambda in l2_lambdas:
   
    #Create a unique directory for each run
    log_dir = "/Users/larasousa/Desktop/uni/ACII/Project_1/best_individual_hiperparameters_13/" + f"run_test{l2_lambda}_{int(time.time())}"
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Pass the tensorboard callback to your training function
    test_accuracy, history = best_hiperparameter_test("mlp", 13, n_epochs, batch_size, learning_rate, momentum, l2_lambda, n_hidden_units, tensorboard_callback)

    # Update best score, parameters, and history if current run is better
    if test_accuracy > best_score:
        best_score = test_accuracy
        best_params = {'l2_lambda': l2_lambda}
        best_history = history



print("The best Hyperparameters: ", best_params)
print("The best score:", best_score)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 256)               2499584   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 10)                2

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 256)               2499584   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 10)                2

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 256)               2499584   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 10)                2

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### The best number of epochs for MLP with mfccs=25

In [18]:


best_score = 0
best_params = {}
best_history = None

for n_epochs in epochs_list:
   
    #Create a unique directory for each run
    log_dir = "/Users/larasousa/Desktop/uni/ACII/Project_1/best_individual_hiperparameters_25/" + f"run_test_{n_epochs}_{int(time.time())}"
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Pass the tensorboard callback to your training function
    test_accuracy, history = best_hiperparameter_test("mlp", 25, n_epochs, batch_size, learning_rate, momentum, l2_lambda, n_hidden_units, tensorboard_callback)

    # Update best score, parameters, and history if current run is better
    if test_accuracy > best_score:
        best_score = test_accuracy
        best_params = {'n_epochs': n_epochs}
        best_history = history


print("The best Hyperparameters: ", best_params)
print("The best score:", best_score)


                 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 18775)             0         
                                                                 
 dense (Dense)               (None, 256)               4806656   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 10)                2

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 18775)             0         
                                                                 
 dense (Dense)               (None, 256)               4806656   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 10)                2

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 18775)             0         
                                                                 
 dense (Dense)               (None, 256)               4806656   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 10)                2

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### The best batch size for MLP with mfccs=25

In [19]:


best_score = 0
best_params = {}
best_history = None

for batch_size in batch_size_list:
   
    #Create a unique directory for each run
    log_dir = "/Users/larasousa/Desktop/uni/ACII/Project_1/best_individual_hiperparameters_25/" + f"run_test{batch_size}_{int(time.time())}"
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Pass the tensorboard callback to your training function
    test_accuracy, history = best_hiperparameter_test("mlp", 25, n_epochs, batch_size, learning_rate, momentum, l2_lambda, n_hidden_units, tensorboard_callback)

    # Update best score, parameters, and history if current run is better
    if test_accuracy > best_score:
        best_score = test_accuracy
        # best_params = {'n_epochs': n_epochs}
        best_params = {'batch_size': batch_size}
        best_history = history



print("The best Hyperparameters: ", best_params)
print("The best score:", best_score)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 18775)             0         
                                                                 
 dense (Dense)               (None, 256)               4806656   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 10)                2

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 18775)             0         
                                                                 
 dense (Dense)               (None, 256)               4806656   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 10)                2

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 18775)             0         
                                                                 
 dense (Dense)               (None, 256)               4806656   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 10)                2

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### The best learning rate for MLP with mfccs=25

In [20]:


best_score = 0
best_params = {}
best_history = None

for learning_rate in learning_rates:
   
    #Create a unique directory for each run
    log_dir = "/Users/larasousa/Desktop/uni/ACII/Project_1/best_individual_hiperparameters_25/" + f"run_test{learning_rate}_{int(time.time())}"
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Pass the tensorboard callback to your training function
    test_accuracy, history = best_hiperparameter_test("mlp", 25, n_epochs, batch_size, learning_rate, momentum, l2_lambda, n_hidden_units, tensorboard_callback)

    # Update best score, parameters, and history if current run is better
    if test_accuracy > best_score:
        best_score = test_accuracy
        # best_params = {'n_epochs': n_epochs}
        best_params = {'learning_rate': learning_rate}
        best_history = history



print("The best Hyperparameters: ", best_params)
print("The best score:", best_score)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 18775)             0         
                                                                 
 dense (Dense)               (None, 256)               4806656   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 10)                2

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 18775)             0         
                                                                 
 dense (Dense)               (None, 256)               4806656   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 10)                2

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 18775)             0         
                                                                 
 dense (Dense)               (None, 256)               4806656   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 10)                2

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### The best Momentum for MLP with mfccs=25

In [21]:

best_score = 0
best_params = {}
best_history = None

for momentum in momentums:
   
    #Create a unique directory for each run
    log_dir = "/Users/larasousa/Desktop/uni/ACII/Project_1/best_individual_hiperparameters_25/" + f"run_test{momentum}_{int(time.time())}"
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Pass the tensorboard callback to your training function
    test_accuracy, history = best_hiperparameter_test("mlp", 25, n_epochs, batch_size, learning_rate, momentum, l2_lambda, n_hidden_units, tensorboard_callback)

    # Update best score, parameters, and history if current run is better
    if test_accuracy > best_score:
        best_score = test_accuracy
        best_params = {'momentum': momentum}
        best_history = history


print("The best Hyperparameters: ", best_params)
print("The best score:", best_score)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 18775)             0         
                                                                 
 dense (Dense)               (None, 256)               4806656   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 10)                2

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 18775)             0         
                                                                 
 dense (Dense)               (None, 256)               4806656   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 10)                2

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 18775)             0         
                                                                 
 dense (Dense)               (None, 256)               4806656   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 10)                2

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### The best Number of Hidden Units for MLP with mfccs=25

In [22]:

best_score = 0
best_params = {}
best_history = None

for n_hidden_units in hidden_units_list:
   
    #Create a unique directory for each run
    log_dir = "/Users/larasousa/Desktop/uni/ACII/Project_1/best_individual_hiperparameters_25/" + f"run_test{n_hidden_units}_{int(time.time())}"
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Pass the tensorboard callback to your training function
    test_accuracy, history = best_hiperparameter_test("mlp", 25, n_epochs, batch_size, learning_rate, momentum, l2_lambda, n_hidden_units, tensorboard_callback)

    # Update best score, parameters, and history if current run is better
    if test_accuracy > best_score:
        best_score = test_accuracy
        best_params = {'n_hidden_units': n_hidden_units}
        best_history = history


print("The best Hyperparameters: ", best_params)
print("The best score:", best_score)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 18775)             0         
                                                                 
 dense (Dense)               (None, 64)                1201664   
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 10)                6

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 18775)             0         
                                                                 
 dense (Dense)               (None, 128)               2403328   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 10)                1

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 18775)             0         
                                                                 
 dense (Dense)               (None, 256)               4806656   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 10)                2

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### RNN

#### The best number of epochs for RNN with mfccs=13

###### Because of the amount of time rnn takes, we will only use epochs of 5, in future work we would adjust this to at least 10

In [23]:
n_epochs = 5

In [24]:


best_score = 0
best_params = {}
best_history = None

for n_epochs in epochs_list:
   
    #Create a unique directory for each run
    log_dir = "/Users/larasousa/Desktop/uni/ACII/Project_1/best_individual_hiperparameters_13_rnn/" + f"run_test_{n_epochs}_{int(time.time())}"
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Pass the tensorboard callback to your training function
    test_accuracy, history = best_hiperparameter_test("rnn", 13, n_epochs, batch_size, learning_rate, momentum, l2_lambda, n_hidden_units, tensorboard_callback)

    # Update best score, parameters, and history if current run is better
    if test_accuracy > best_score:
        best_score = test_accuracy
        # best_params = {'n_epochs': n_epochs}
        best_params = {'n_epochs': n_epochs}
        best_history = history


print("The best Hyperparameters: ", best_params)
print("The best score:", best_score)


                 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 751, 256)          276480    
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 dense (Dense)               (None, 10)                2570      
                                                                 
Total params: 804362 (3.07 MB)
Trainable params: 804362 (3.07 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
48/48 [==============================] - 575s 12s/step - loss: 6.8249 - accuracy: 0.0513 - val_loss: 6.7436 - val_accuracy: 0.0662
Epoch 2/5
48/48 [==============================] - 608s 13s/step - loss: 6.6629 - accuracy: 0.0855 - val_loss: 6.5650 - val_accuracy: 0.1881
Ep

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 751, 256)          276480    
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 dense (Dense)               (None, 10)                2570      
                                                                 
Total params: 804362 (3.07 MB)
Trainable params: 804362 (3.07 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
48/48 [==============================] - 576s 12s/step - loss: 6.8059 - accuracy: 0.1479 - val_loss: 6.7282 - val_accuracy: 0.1660
Epoch 2/10
48/48 [==============================] - 615s 13s/step - loss: 6.6451 - accuracy: 0.1787 - val_loss: 6.5519 - val_accuracy: 0.1671


/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 751, 256)          276480    
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 dense (Dense)               (None, 10)                2570      
                                                                 
Total params: 804362 (3.07 MB)
Trainable params: 804362 (3.07 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/15
48/48 [==============================] - 606s 13s/step - loss: 6.8075 - accuracy: 0.1175 - val_loss: 6.7314 - val_accuracy: 0.1103
Epoch 2/15
48/48 [==============================] - 684s 14s/step - loss: 6.6460 - accuracy: 0.1170 - val_loss: 6.5551 - val_accuracy: 0.1109


/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### The best batch size for RNN with mfccs=13

### Tthe following cells were not run, as rnn required too much execution time

In [ ]:

best_score = 0
best_params = {}
best_history = None

for batch_size in batch_size_list:
   
    #Create a unique directory for each run
    log_dir = "/Users/larasousa/Desktop/uni/ACII/Project_1/best_individual_hiperparameters_13_rnn/" + f"run_test{batch_size}_{int(time.time())}"
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Pass the tensorboard callback to your training function
    test_accuracy, history = best_hiperparameter_test("rnn", 13, n_epochs, batch_size, learning_rate, momentum, l2_lambda, n_hidden_units, tensorboard_callback)

    # Update best score, parameters, and history if current run is better
    if test_accuracy > best_score:
        best_score = test_accuracy
        best_params = {'batch_size': batch_size}
        best_history = history


print("The best Hyperparameters: ", best_params)
print("The best score:", best_score)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 128)               1249792   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 10)                1

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 128)               1249792   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 10)                1

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 128)               1249792   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 10)                1

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### The best learning rate for RNN with mfccs=13

In [25]:

best_score = 0
best_params = {}
best_history = None

for learning_rate in learning_rates:
   
    #Create a unique directory for each run
    log_dir = "/Users/larasousa/Desktop/uni/ACII/Project_1/best_individual_hiperparameters_13_rnn/" + f"run_test{learning_rate}_{int(time.time())}"
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Pass the tensorboard callback to your training function
    test_accuracy, history = best_hiperparameter_test("rnn", 13, n_epochs, batch_size, learning_rate, momentum, l2_lambda, n_hidden_units, tensorboard_callback)

    # Update best score, parameters, and history if current run is better
    if test_accuracy > best_score:
        best_score = test_accuracy
        best_params = {'learning_rate': learning_rate}
        best_history = history


print("The best Hyperparameters: ", best_params)
print("The best score:", best_score)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 751, 256)          276480    
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 dense (Dense)               (None, 10)                2570      
                                                                 
Total params: 804362 (3.07 MB)
Trainable params: 804362 (3.07 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/15
48/48 [==============================] - 698s 14s/step - loss: 6.8519 - accuracy: 0.1246 - val_loss: 6.8420 - val_accuracy: 0.1202
Epoch 2/15
48/48 [==============================] - 714s 15s/step - loss: 6.8351 - accuracy: 0.1249 - val_loss: 6.8232 - val_accuracy: 0.1169


KeyboardInterrupt: 

#### The best Momentum for RNN with mfccs=13

In [ ]:

best_score = 0
best_params = {}
best_history = None

for momentum in momentums:
   
    #Create a unique directory for each run
    log_dir = "/Users/larasousa/Desktop/uni/ACII/Project_1/best_individual_hiperparameters_13_rnn/" + f"run_test{momentum}_{int(time.time())}"
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Pass the tensorboard callback to your training function
    test_accuracy, history = best_hiperparameter_test("rnn", 13, n_epochs, batch_size, learning_rate, momentum, l2_lambda, n_hidden_units, tensorboard_callback)

    # Update best score, parameters, and history if current run is better
    if test_accuracy > best_score:
        best_score = test_accuracy
        best_params = {'momentum': momentum}
        best_history = history


print("The best Hyperparameters: ", best_params)
print("The best score:", best_score)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 128)               1249792   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 10)                1

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 128)               1249792   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 10)                1

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/15
48/48 [==============================] - 2s 23ms/step - loss: 2.3621 - accuracy: 0.1133 - val_loss: 2.3408 - val_accuracy: 0.1131
Epoch 2/15
48/48 [==============================] - 1s 14ms/step - loss: 2.3467 - accuracy: 0.1160 - val_loss: 2.3296 - val_accuracy: 0.1114
Epoch 3/15
48/48 [==============================] - 1s 16ms/step - loss: 2.3358 - accuracy: 0.1160 - val_loss: 2.3214 - val_accuracy: 0.1103
Epoch 4/15
48/48 [==============================] - 1s 18ms/step - loss: 2.3278 - accuracy: 0.1161 - val_loss: 2.3159 - val_accuracy: 0.1103
Epoch 5/15
48/48 [==============================] - 1s 16ms/step - loss: 2.3223 - accuracy: 0.1161 - val_loss: 2.3124 - val_accuracy: 0.1103
Epoch 6/15
48/48 [==============================] - 1s 17ms/step - loss: 2.3188 - accuracy: 0.1161 - val_loss: 2.3102 - val_accuracy: 0.1103
Epoch 7/15
48/48 [==============================] - 1s 17ms/step - loss: 2.3167 - accuracy: 0.1161 - val_loss: 2.3087 - val_accuracy: 0.1103
Epoch 8/15
48

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### The best Number of Hidden Units for RNN with mfccs=13

In [ ]:

best_score = 0
best_params = {}
best_history = None

for n_hidden_units in hidden_units_list:
   
    #Create a unique directory for each run
    log_dir = "/Users/larasousa/Desktop/uni/ACII/Project_1/best_individual_hiperparameters_13_rnn/" + f"run_test{n_hidden_units}_{int(time.time())}"
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Pass the tensorboard callback to your training function
    test_accuracy, history = best_hiperparameter_test("rnn", 13, n_epochs, batch_size, learning_rate, momentum, l2_lambda, n_hidden_units, tensorboard_callback)

    # Update best score, parameters, and history if current run is better
    if test_accuracy > best_score:
        best_score = test_accuracy
        best_params = {'n_hidden_units': n_hidden_units}
        best_history = history


print("The best Hyperparameters: ", best_params)
print("The best score:", best_score)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 64)                624896    
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 10)                6

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 128)               1249792   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 10)                1

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/15
48/48 [==============================] - 2s 35ms/step - loss: 2.4343 - accuracy: 0.1170 - val_loss: 2.4505 - val_accuracy: 0.1103
Epoch 2/15
48/48 [==============================] - 1s 25ms/step - loss: 2.4337 - accuracy: 0.1158 - val_loss: 2.4263 - val_accuracy: 0.1103
Epoch 3/15
48/48 [==============================] - 1s 23ms/step - loss: 2.4128 - accuracy: 0.1158 - val_loss: 2.4065 - val_accuracy: 0.1103
Epoch 4/15
48/48 [==============================] - 1s 23ms/step - loss: 2.3973 - accuracy: 0.1158 - val_loss: 2.3916 - val_accuracy: 0.1103
Epoch 5/15
48/48 [==============================] - 1s 23ms/step - loss: 2.3861 - accuracy: 0.1158 - val_loss: 2.3822 - val_accuracy: 0.1103
Epoch 6/15
48/48 [==============================] - 1s 22ms/step - loss: 2.3789 - accuracy: 0.1158 - val_loss: 2.3752 - val_accuracy: 0.1103
Epoch 7/15
48/48 [==============================] - 1s 21ms/step - loss: 2.3735 - accuracy: 0.1158 - val_loss: 2.3707 - val_accuracy: 0.1103
Epoch 8/15
48

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### The best L2_lambda for RNN with mfccs=13

In [ ]:

best_score = 0
best_params = {}
best_history = None

for l2_lambda in l2_lambdas:
   
    #Create a unique directory for each run
    log_dir = "/Users/larasousa/Desktop/uni/ACII/Project_1/best_individual_hiperparameters_13_rnn/" + f"run_test{l2_lambda}_{int(time.time())}"
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Pass the tensorboard callback to your training function
    test_accuracy, history = best_hiperparameter_test("rnn", 13, n_epochs, batch_size, learning_rate, momentum, l2_lambda, n_hidden_units, tensorboard_callback)

    # Update best score, parameters, and history if current run is better
    if test_accuracy > best_score:
        best_score = test_accuracy
        best_params = {'l2_lambda': l2_lambda}
        best_history = history


print("The best Hyperparameters: ", best_params)
print("The best score:", best_score)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 256)               2499584   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 10)                2

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 256)               2499584   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 10)                2

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 256)               2499584   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 10)                2

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### The best number of epochs for RNN with mfccs=25

In [ ]:

best_score = 0
best_params = {}
best_history = None

for n_epochs in epochs_list:
   
    #Create a unique directory for each run
    log_dir = "/Users/larasousa/Desktop/uni/ACII/Project_1/best_individual_hiperparameters_25_rnn/" + f"run_test_{n_epochs}_{int(time.time())}"
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Pass the tensorboard callback to your training function
    test_accuracy, history = best_hiperparameter_test("rnn", 25, n_epochs, batch_size, learning_rate, momentum, l2_lambda, n_hidden_units, tensorboard_callback)

    # Update best score, parameters, and history if current run is better
    if test_accuracy > best_score:
        best_score = test_accuracy
        best_params = {'n_epochs': n_epochs}
        best_history = history


print("The best Hyperparameters: ", best_params)
print("The best score:", best_score)


                 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 18775)             0         
                                                                 
 dense (Dense)               (None, 256)               4806656   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 10)                2

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 18775)             0         
                                                                 
 dense (Dense)               (None, 256)               4806656   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 10)                2

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 18775)             0         
                                                                 
 dense (Dense)               (None, 256)               4806656   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 10)                2

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### The best batch size for RNN with mfccs=25

In [ ]:

best_score = 0
best_params = {}
best_history = None

for batch_size in batch_size_list:
   
    #Create a unique directory for each run
    log_dir = "/Users/larasousa/Desktop/uni/ACII/Project_1/best_individual_hiperparameters_25_rnn/" + f"run_test{batch_size}_{int(time.time())}"
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Pass the tensorboard callback to your training function
    test_accuracy, history = best_hiperparameter_test("rnn", 25, n_epochs, batch_size, learning_rate, momentum, l2_lambda, n_hidden_units, tensorboard_callback)

    # Update best score, parameters, and history if current run is better
    if test_accuracy > best_score:
        best_score = test_accuracy
        best_params = {'batch_size': batch_size}
        best_history = history


print("The best Hyperparameters: ", best_params)
print("The best score:", best_score)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 18775)             0         
                                                                 
 dense (Dense)               (None, 256)               4806656   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 10)                2

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 18775)             0         
                                                                 
 dense (Dense)               (None, 256)               4806656   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 10)                2

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 18775)             0         
                                                                 
 dense (Dense)               (None, 256)               4806656   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 10)                2

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### The best learning rate for RNN with mfccs=25

In [ ]:

best_score = 0
best_params = {}
best_history = None

for learning_rate in learning_rates:
   
    #Create a unique directory for each run
    log_dir = "/Users/larasousa/Desktop/uni/ACII/Project_1/best_individual_hiperparameters_25_rnn/" + f"run_test{learning_rate}_{int(time.time())}"
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Pass the tensorboard callback to your training function
    test_accuracy, history = best_hiperparameter_test("rnn", 25, n_epochs, batch_size, learning_rate, momentum, l2_lambda, n_hidden_units, tensorboard_callback)

    # Update best score, parameters, and history if current run is better
    if test_accuracy > best_score:
        best_score = test_accuracy
        best_params = {'learning_rate': learning_rate}
        best_history = history


print("The best Hyperparameters: ", best_params)
print("The best score:", best_score)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 128)               1249792   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 10)                1

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 128)               1249792   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 10)                1

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 128)               1249792   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 10)                1

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### The best Momentum for RNN with mfccs=25

In [ ]:

best_score = 0
best_params = {}
best_history = None

for momentum in momentums:
   
    #Create a unique directory for each run
    log_dir = "/Users/larasousa/Desktop/uni/ACII/Project_1/best_individual_hiperparameters_25_rnn/" + f"run_test{momentum}_{int(time.time())}"
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Pass the tensorboard callback to your training function
    test_accuracy, history = best_hiperparameter_test("rnn", 25, n_epochs, batch_size, learning_rate, momentum, l2_lambda, n_hidden_units, tensorboard_callback)

    # Update best score, parameters, and history if current run is better
    if test_accuracy > best_score:
        best_score = test_accuracy
        best_params = {'momentum': momentum}
        best_history = history


print("The best Hyperparameters: ", best_params)
print("The best score:", best_score)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 128)               1249792   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 10)                1

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 128)               1249792   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 10)                1

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/15
48/48 [==============================] - 2s 23ms/step - loss: 2.3621 - accuracy: 0.1133 - val_loss: 2.3408 - val_accuracy: 0.1131
Epoch 2/15
48/48 [==============================] - 1s 14ms/step - loss: 2.3467 - accuracy: 0.1160 - val_loss: 2.3296 - val_accuracy: 0.1114
Epoch 3/15
48/48 [==============================] - 1s 16ms/step - loss: 2.3358 - accuracy: 0.1160 - val_loss: 2.3214 - val_accuracy: 0.1103
Epoch 4/15
48/48 [==============================] - 1s 18ms/step - loss: 2.3278 - accuracy: 0.1161 - val_loss: 2.3159 - val_accuracy: 0.1103
Epoch 5/15
48/48 [==============================] - 1s 16ms/step - loss: 2.3223 - accuracy: 0.1161 - val_loss: 2.3124 - val_accuracy: 0.1103
Epoch 6/15
48/48 [==============================] - 1s 17ms/step - loss: 2.3188 - accuracy: 0.1161 - val_loss: 2.3102 - val_accuracy: 0.1103
Epoch 7/15
48/48 [==============================] - 1s 17ms/step - loss: 2.3167 - accuracy: 0.1161 - val_loss: 2.3087 - val_accuracy: 0.1103
Epoch 8/15
48

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### The best Number of Hidden Units for RNN with mfccs=25

In [ ]:

best_score = 0
best_params = {}
best_history = None

for n_hidden_units in hidden_units_list:
   
    #Create a unique directory for each run
    log_dir = "/Users/larasousa/Desktop/uni/ACII/Project_1/best_individual_hiperparameters_25_rnn/" + f"run_test{n_hidden_units}_{int(time.time())}"
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Pass the tensorboard callback to your training function
    test_accuracy, history = best_hiperparameter_test("rnn", 25, n_epochs, batch_size, learning_rate, momentum, l2_lambda, n_hidden_units, tensorboard_callback)

    # Update best score, parameters, and history if current run is better
    if test_accuracy > best_score:
        best_score = test_accuracy
        best_params = {'n_hidden_units': n_hidden_units}
        best_history = history


print("The best Hyperparameters: ", best_params)
print("The best score:", best_score)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 64)                624896    
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 10)                6

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 128)               1249792   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 10)                1

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/15
48/48 [==============================] - 2s 35ms/step - loss: 2.4343 - accuracy: 0.1170 - val_loss: 2.4505 - val_accuracy: 0.1103
Epoch 2/15
48/48 [==============================] - 1s 25ms/step - loss: 2.4337 - accuracy: 0.1158 - val_loss: 2.4263 - val_accuracy: 0.1103
Epoch 3/15
48/48 [==============================] - 1s 23ms/step - loss: 2.4128 - accuracy: 0.1158 - val_loss: 2.4065 - val_accuracy: 0.1103
Epoch 4/15
48/48 [==============================] - 1s 23ms/step - loss: 2.3973 - accuracy: 0.1158 - val_loss: 2.3916 - val_accuracy: 0.1103
Epoch 5/15
48/48 [==============================] - 1s 23ms/step - loss: 2.3861 - accuracy: 0.1158 - val_loss: 2.3822 - val_accuracy: 0.1103
Epoch 6/15
48/48 [==============================] - 1s 22ms/step - loss: 2.3789 - accuracy: 0.1158 - val_loss: 2.3752 - val_accuracy: 0.1103
Epoch 7/15
48/48 [==============================] - 1s 21ms/step - loss: 2.3735 - accuracy: 0.1158 - val_loss: 2.3707 - val_accuracy: 0.1103
Epoch 8/15
48

/Users/larasousa/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
